The most important key building block in `LangChain` is the `chain`.
A `chain` usually combines an LLM with a prompt, but also allows us to combine other chains and perform several operations on text.
One of the powers of a `chain` is running it over many inputs at a time.

# `LLMChain`

The `LLMChain` is a simple, but powerful chain that underpins a lot of the chains covered in this lesson.

In [1]:
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
api_version = "2023-12-01-preview"
model = "gpt-35-turbo-16k"

We initialize our model with a high temperature to elicit more randomness ("creativity") in the response.

In [3]:
chat = AzureChatOpenAI(model=model, temperature=0.9, api_version=api_version)

In [4]:
prompt = ChatPromptTemplate.from_template(
    template="What is the best name to describe a company that makes {product}?"
)

We combine the prompt and the model using the `LLMChain`.

In [5]:
chain = LLMChain(prompt=prompt, llm=chat)

We can run the chain by providing an input to the `run` method.

In [6]:
product = "Queen Size Sheet Set"
chain.run(product)

'"Regal Rest Linens"'

# `SimpleSequentialChain`

The `LLMChain` is the most simple, basic type of chain in `LangChain`.
The `SimpleSequentialChain` runs a sequence of chains one after another.
This works well when we have chains that expect one input and return one output.

In [7]:
from langchain.chains import SimpleSequentialChain

To demonstrate, we'll treat our prior `LLMChain` as our first chain in a sequence of chains.

In [8]:
chain_one = chain.copy()

Now we create a second chain to take the output of the first chain (`chain_one`), a company name, and pass it into a second chain.

In [9]:
second_prompt = ChatPromptTemplate.from_template(
    template="Write a 20 word description for the following company: {company_name}"
)

chain_two = LLMChain(prompt=second_prompt, llm=chat)

We supply both chains to `SimpleSequentialChain` in the order we want to run.

In [10]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

We run the `simple_chain` like any other chain using the `run` method.
Remember that we are inputting a product as that is what the first chain (`chain_one`) expects.

In [11]:
simple_chain.run(product)



> Entering new SimpleSequentialChain chain...


AttributeError: 'LLMChain' object has no attribute 'callbacks'

Nearly 200 versions of `LangChain` have come out since the the DeepLearningAI tutorial was created.
After some searching on GitHub and StackOverflow, I haven't found a great answer to get around the following error:

```python
AttributeError: 'LLMChain' object has no attribute 'callbacks'
```

I could create different chains based on some prior experience, but the type would be `RunnableSequence` instead of `LLMChain`.

In [12]:
type(chain)

langchain.chains.llm.LLMChain

In [13]:
chain_one = prompt | chat

In [14]:
type(chain_one)

langchain_core.runnables.base.RunnableSequence

In [15]:
chain_two = second_prompt | chat

In [16]:
from langchain_core.runnables.base import RunnableSequence

In [17]:
RunnableSequence?

Init signature:
RunnableSequence(
    *steps: 'RunnableLike',
    name: 'Optional[str]' = None,
    first: 'Optional[Runnable[Any, Any]]' = None,
    middle: 'Optional[List[Runnable[Any, Any]]]' = None,
    last: 'Optional[Runnable[Any, Any]]' = None,
) -> None
Docstring:     
A sequence of runnables, where the output of each is the input of the next.

RunnableSequence is the most important composition operator in LangChain as it is
used in virtually every chain.

A RunnableSequence can be instantiated directly or more commonly by using the `|`
operator where either the left or right operands (or both) must be a Runnable.

Any RunnableSequence automatically supports sync, async, batch.

The default implementations of `batch` and `abatch` utilize threadpools and
asyncio gather and will be faster than naive invocation of invoke or ainvoke
for IO bound runnables.

Batching is implemented by invoking the batch method on each component of the
RunnableSequence in order.

A RunnableSequence p

In [18]:
simple_chain = chain_one | chain_two

In [20]:
hasattr(simple_chain, "run")

False

A `RunnableSequence` doesn't have a `run` method, so we use `invoke` instead.

In [21]:
simple_chain.invoke(input={"product": "product"})

TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'langchain_core.messages.ai.AIMessage'>.

Executing this raises the following:

```python
TypeError: Expected mapping type as input to ChatPromptTemplate. Received <class 'langchain_core.messages.ai.AIMessage'>.
```

This is do to `chain_one` returning an `AIMessage`.

In [22]:
output_one = chain_one.invoke(input={"product": product})
print(type(output_one))

<class 'langchain_core.messages.ai.AIMessage'>


`chain_two` expects a mapping type as input, i.e. a dictionary with the `input_variables` as keys and the necessay values.

In [23]:
chain_two.first.input_variables

['company_name']

To get this to work, I need to convert the output of `chain_one` into the required input type for `chain_two`.
I can do this with a `RunnableLambda`.

In [31]:
from langchain_core.runnables import RunnableLambda

This is what I want to return.

In [60]:
expect_output = {"company_name": output_one.content}
expect_output

{'company_name': '"Regal Rest Linens"'}

In [59]:
runnable = {"company_name": RunnableLambda(func=lambda x: x.content)}
runnable

{'company_name': RunnableLambda(lambda x: x.content)}

Adding `runnable` to the end of `chain_one`, we can test if the result is as expected.

In [54]:
test_chain = chain_one | runnable

In [62]:
test_chain_output = test_chain.invoke(input={"product": product})
test_chain_output

{'company_name': 'Royal Comfort Linens'}

Testing as input to `chain_two`.

In [57]:
chain_two.invoke(input=test_chain_output)

AIMessage(content='RoyalRest is a luxury bedding company that provides exquisite and comfortable sleep experiences fit for royalty.')

Looks good to me!
Let's combine everything together.

In [63]:
simple_chain = chain_one | runnable | chain_two
simple_chain.invoke(input={"product": product})

AIMessage(content='RoyalLux Beddings is a luxury bedding company offering exquisite and comfortable linens, pillows, and blankets fit for royalty.')